In [114]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    extract_images_in_survival_order,
)

from src.plots import (
    plot_observation,
    plot_deeplab_mobile_predictions,
    plot_mobile_prediction_from_path
)

from src.deeplab_mobile.modelling import(
    select_images_input,
    get_deeplab_mobile_model,
    train_deeplab_mobile
)

from src.deeplab_mobile.segdataset import(
    get_mobile_dataloaders
)

from src.utils import(
    LOGS_FILE_PATH,
    TYPE_NAMES
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
data_path = Path(os.getcwd()) / 'data' / 'HGG'
survival = pd.read_csv(data_path.parent /'survival_data.csv')

In [116]:
dir_ids = survival['BraTS19ID']

In [117]:
t2, t1ce, t1, flair, seg = extract_images_in_survival_order(data_path, dir_ids)
images = [t2, t1ce, t1, flair, seg]

In [118]:
for i in range(len(TYPE_NAMES)):
    survival[TYPE_NAMES[i]] = images[i] 

In [119]:
modelname = 'flair_totalpipe_old.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)
model.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1),

In [120]:
flair_segout = []

for i in range(len(survival)):
    input_tensor = torch.tensor(survival['flair'][i]).expand(3, -1, -1).type(torch.ShortTensor).float().unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0][0]
    flair_segout.append(np.array(output))

survival['flair_seg'] = flair_segout

In [121]:
survival = survival.dropna().reset_index(drop=True)
survival['Survival'] = survival['Survival'].apply(lambda x: str(x))

In [122]:
to_drop = []
for i in range(len(survival)):
    if 'ALIVE' in survival['Survival'].loc[i]:
        to_drop.append(i)
        
        
survival.drop(to_drop, inplace=True)
print(len(survival))

103


In [123]:
survival['Survival'] = survival['Survival'].apply(lambda x: int(x))

In [124]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(51984, 1200)
        self.fc2 = nn.Linear(1200, 340)
        self.fc3 = nn.Linear(340, 30)
        self.fc4 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net()

In [125]:
from torch.utils.data import Dataset

class Survival_Dataset(Dataset):
    def __init__(self,
                 images,
                 targets,
                 fraction = 0.1,
                 subset = None,
                 image_color_mode = "rgb"
                 ) -> None:

        if image_color_mode not in ["rgb", "grayscale"]:
            raise ValueError(
                f"{image_color_mode} is an invalid choice. Please enter from rgb grayscale."
            )

        self.image_color_mode = image_color_mode

        if subset not in ["Train", "Test"]:
            raise (ValueError(
                f"{subset} is not a valid input. Acceptable values are Train and Test."
            ))

        self.fraction = fraction
        self.images = images
        self.targets = targets

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, index: int):
        image = self.images[index]
        target = self.targets[index]
        
        sample = {"image": image, "target": target}
        sample["image"] = torch.tensor(sample["image"]).expand(3, -1, -1).type(torch.ShortTensor)
        sample["target"] = target
        return sample

In [126]:
def get_survival_dataloaders(images, target, batch_size=14):
    image_datasets = {
        x: Survival_Dataset(
            images,
            target,
            subset=x)
        for x in ['Train', 'Test']
    }

    dataloaders = {
            x: DataLoader(image_datasets[x],
                        batch_size=batch_size,
                        )
            for x in ['Train', 'Test']
        }
    return dataloaders

In [127]:
dataloaders = get_survival_dataloaders(survival['flair'].values, survival['Survival'].values)

In [136]:
import time
import copy
from sklearn.metrics import f1_score
from tqdm import tqdm

num_epochs = 2
metrics={'f1_score': f1_score}
running_loss = 0.0
best_loss = 1e10

since = time.time()
best_model_wts = copy.deepcopy(net.state_dict())


fieldnames = ['epoch', 'Train_loss', 'Test_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Test_{m}' for m in metrics.keys()]

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, num_epochs+1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)

    batchsummary = {a: [0] for a in fieldnames}

    for phase in ['Train', 'Test']:
        if phase == 'Train':
            net.train()
        else:
            net.eval()

        for sample in tqdm(iter(dataloaders[phase])):
            inputs = sample['image'].float()
            targets = sample['target'].float()
            # zero the parameter gradients
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'Train'):
                outputs = net(inputs).squeeze()
                print(outputs.squeeze().shape)
                print(targets.shape)
                loss = criterion(outputs, targets)
                print(loss)
                y_pred = outputs.data.cpu().numpy().ravel()
                y_true = targets.data.cpu().numpy().ravel()
                for name, metric in metrics.items():
                    if name == 'f1_score':
                        # Use a classification threshold of 0.1
                        batchsummary[f'{phase}_{name}'].append(metric(y_true > 0, y_pred > 0.1))
                    else:
                        batchsummary[f'{phase}_{name}'].append(metric(y_true.astype('uint8'), y_pred))

                if phase == 'Train':
                    loss.backward()
                    optimizer.step()
        batchsummary['epoch'] = epoch
        epoch_loss = loss
        batchsummary[f'{phase}_loss'] = epoch_loss.item()
        print('{} Loss: {:.4f}'.format(phase, loss))
    for field in fieldnames[3:]:
        batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
            # deep copy the model
        if phase == 'Test' and loss < best_loss:
            best_loss = loss
            best_model_wts = copy.deepcopy(net.state_dict())

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
time_elapsed // 60, time_elapsed % 60))
print('Lowest Loss: {:4f}'.format(best_loss))

# load best model weights
net.load_state_dict(best_model_wts)

Epoch 1/2
----------


  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([14])
torch.Size([14])
tensor(319872.5625, grad_fn=<MseLossBackward0>)


 12%|█▎        | 1/8 [00:00<00:05,  1.35it/s]

torch.Size([14])
torch.Size([14])
tensor(1.3642e+33, grad_fn=<MseLossBackward0>)


...

 75%|███████▌  | 6/8 [00:01<00:00,  4.35it/s]

torch.Size([14])
torch.Size([14])
tensor(nan)


100%|██████████| 8/8 [00:01<00:00,  4.49it/s]

torch.Size([14])
torch.Size([14])
tensor(nan)
torch.Size([5])
torch.Size([5])
tensor(nan)
Test Loss: nan
{'epoch': 2, 'Train_loss': nan, 'Test_loss': nan, 'Train_f1_score': 0.0, 'Test_f1_score': [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}
{'epoch': 2, 'Train_loss': nan, 'Test_loss': nan, 'Train_f1_score': 0.0, 'Test_f1_score': 0.0}
Training complete in 0m 17s
Lowest Loss: 10000000000.000000


<All keys matched successfully>

In [135]:
net()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [6, 3, 5, 5], but got 2-dimensional input of size [240, 240] instead

In [ ]:
survival